In [1]:
import pandas_datareader.data as reader
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import statsmodels.api as sm


In [ ]:
# CAPM : r_i = alpha + beta * r_m + e
